## <font color=#B2D732> <span style="background-color: #4424D6"> Imports

In [1]:
import sys,json
import glob, os
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/code/")

from seed_to_voxels_inprogress_nk import Seed2voxels

%matplotlib inline
%load_ext autoreload
%autoreload 2

/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/nilearn/__init__.py:69: FutureWarning: Python 3.6 support is deprecated and will be removed in release 0.10 of Nilearn. Consider switching to Python 3.8 or 3.9.
  _python_deprecation_warnings()


## <font color=#B2D732> <span style="background-color: #4424D6"> Initialization

 ### <font color=#4424D6> I. Configuration & parameters </font >

In [22]:
with open('../config/config_CL.json') as config_file:
    config = json.load(config_file) # load config file
signal='raw'
seed_names=['brain_L-BA4'] # define seed names ex: 'spinalcord_ICA-C4'
target_name=['brain_mask'] # define structure target


### <font color=#4424D6> II. Select files </font>

In [36]:
# One target per subject
data_target=[];ts_target_txt=[];ts_seed_txt = [];data_seed=[];
seed_name = seed_names[0]
for subject_name in config['list_subjects']:
    data_target.append(glob.glob(config["coreg_dir"] + 'sub-'+ subject_name +'/'+target_name[0].split('_')[0]+'/*' + config["coreg_tag"][target_name[0].split('_')[0]] +'*.gz')[0])
    ts_target_txt.append(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+target_name[0]+'/timeseries/sub_' + subject_name + '_mask_' + target_name[0] + '_timeseries')
    
    data_seed.append(glob.glob(config["coreg_dir"] + 'sub-'+ subject_name +'/'+seed_name.split('_')[0]+'/*' + config["coreg_tag"][target_name[0].split('_')[0]] +'*.gz')[0])
    ts_seed_txt.append(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+seed_name+'/timeseries/sub_' + subject_name + '_mask_' + seed_name.split('_')[-1] + '_timeseries')

mask_seed=glob.glob(config["main_dir"] + config["templates"][seed_name.split('_')[0]+"_seeds"] + seed_name.split('_')[-1].split('_')[-1] +'*.gz')[0]
mask_target=glob.glob(config["main_dir"] + config["masks"][target_name[0].split('_')[0]])[0]

    
# create output directory if needed
if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+seed_name):
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+seed_name)
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+seed_name+'/timeseries/') # folder to store timeseries extraction
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/'+seed_name+'/brain_fc_maps/') # folder to store maps of FC

In [35]:
data_seed

['/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/func_preparation/2_func_in_template/sub-P028/brain/sub-P028_task-rest_bold_stc_brain_moco_denoised_reg_SPM_inTemplate.nii.gz',
 '/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/func_preparation/2_func_in_template/sub-P030/brain/sub-P030_task-rest_bold_stc_brain_moco_denoised_reg_SPM_inTemplate.nii.gz',
 '/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/func_preparation/2_func_in_template/sub-P033/brain/sub-P033_task-rest_bold_stc_brain_moco_denoised_reg_SPM_inTemplate.nii.gz',
 '/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/func_preparation/2_func_in_template/sub-P043/brain/sub-P043_task-rest_bold_stc_brain_moco_denoised_reg_SPM_inTemplate.nii.gz',
 '/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/func_preparation/2_func_in_template/sub-P047/brain/sub-P047_task-rest_bold_stc_brain_moco_denoised_reg_SPM_inTemplate.nii.gz',
 '/cerebro/cerebro1/

### <font color=#4424D6> III. Initialize class based on this </font>

In [37]:
seed2voxels=Seed2voxels(config,signal,seed_names,target_name) # initialize the function

## <font color=#B2D732> <span style="background-color: #4424D6"> Data extraction </font></span>
### <font color=#4424D6> I. Time series extraction - Seed </font>

In [38]:
seed_timeseries=[]
seed_timeseries,seed_timeseries_mean,seed_timeseries_pc1=seed2voxels.extract_data(img=data_seed, mask=mask_seed, timeseries_txt=ts_seed_txt,
                                                                run="extract",n_jobs=8) # run the analyse for target voxels

### <font color=#4424D6> II. Time series extraction - Target </font>

In [39]:
target_timeseries,target_timeseries_mean,target_timeseries_pc1=seed2voxels.extract_data(img=data_target, mask=mask_target, timeseries_txt=ts_target_txt,
                                                                run="extract",n_jobs=8,smoothing_fwhm=[6,6,6]) # run the analyse for target voxels


## <font color=#B2D732> <span style="background-color: #4424D6"> Correlation analysis

In [41]:
output_img={};
for seed_name in seed_names:
    output_img[seed_name]=[]
    output_img[seed_name]=config['main_dir'] + config['seed2vox_dir'] + '/1_first_level/' + seed_name + '/' + target_name[0].split('_')[0]+ '_fc_maps/' + str(len(config['list_subjects'])) + 'subjects_seedPC1_' + seed_name.split('_')[-1] + '_s_BP'

seed2voxels.correlation_maps(seed_timeseries_mean,target_timeseries,mask=mask_target,output_img=output_img[seed_name],Fisher=True,n_jobs=8)

## <font color=#B2D732> <span style="background-color: #4424D6"> Smoothing

In [11]:
from nilearn import image
img_smoothed=image.smooth_img(output_img[seed_name],[6,6,6])
img_smoothed.to_filename(output_img[seed_name].split('.')[0] + '_s.nii' )